In [25]:
import sys
sys.path.append('../..')
from sisyphus2.tasks import rdm as rd
from sisyphus2.backend.models.basic import Basic

from matplotlib import pyplot as plt
%matplotlib inline

define params globally first before passing to RDM, mess w/ model params and explain what can do. (can turn on and off dale's law). some masking, input output connectivity. train_params variables stuff

In [26]:
dt = 10 # time-step
tau = 100 # takes step of size(dt/tau) times the descent direction when updating state TODO(jasmine): check this
T = 2000 # time to run for (number of steps is T/dt)
N_batch = 50 # number of trials TODO(jasmine): verify this
N_rec = 50 # number of recurrent units TODO(jasmine):verify this
name = 'basicModel'

In [27]:
rdm = rd.RDM(dt = dt, tau = tau, T = T, N_batch = N_batch)

RDM puts the params passed in as well as other generated params into a dict we can then use to create our model. 

In [28]:
params = rdm.__dict__
print(params)

{'tau': 100, 'N_in': 2, 'N_batch': 50, 'T': 2000, 'N_steps': 200, 'alpha': 0.1, 'dt': 10, 'N_out': 2}


Generate *N_batch* trials to be used in training

In [29]:
gen = rdm.batch_generator()

We add in a few params that Basic(RNN) needs but that RDM doesn't generate for us.

In [30]:
params['name'] = name #Used to scope out a namespace for global variables.
params['N_rec'] = N_rec

There are some other optional parameters we can add in:

In [31]:
params['dale_ratio'] = None # Default: None -- when the dale_ratio is set, dale's law is applied
params['rec_noise'] = 0.0 # Default: 0.0 -- how much noise to add to the new_state calculation
params['W_in_train'] = True # Indicates whether W_in is trainable. Default: True
params['W_rec_train'] = True # Indicates whether W_rec is trainable. Default: True
params['W_out_train'] = True # Indicates whether W_out is trainable. Default: True
params['b_rec_train'] = True # Indicates whether b_rec is trainable. Default: True
params['b_out_train'] = True # Indicates whether b_out is trainable. Default: True
params['init_state_train'] = True # Indicates whether init_state is trainable. Default: True
params['load_weights_path'] = None # When given a path, loads weights from file in that path. Default: None

Instantiate our model

In [32]:
basicModel = Basic(params)

Start a tensorflow session with loss, regularization, predictions, and regularized loss defined.

In [33]:
basicModel.build()

Set the training parameters for our model. All of the parameters below are optional.

In [34]:
train_params = {}
train_params['save_weights_path'] =  '../weights/refactor_weights.npz' # Where to save the model after training. Default: None
train_params['training_iters'] = 1000 # number of iterations to train for Default: 10000
train_params['learning_rate' = .001 # Sets learning rate if use default optimizer Default: .001
train_params['loss_epoch'] = 10 # Compute and record loss every 'loss_epoch' epochs. Default: 10
train_params['verbosity'] = True # If true, prints information as training progresses. Default: True
train_params['save_training_weights_epoch'] = 100 # save training weights every 'save_training_weights_epoch' epochs. Default: 100
train_params['training_weights_path'] = None # where to save training weights as training progresses. Default: None
train_params['generator_function'] = None # replaces trial_batch_generator with the generator_function when not none. Default: None
train_params['optimizer'] = tf.train.AdamOptimizer(learning_rate=train_params['learning_rate']) # What optimizer to use to compute gradients. Default: tf.train.AdamOptimizer(learning_rate=train_params['learning_rate'])
train_params['clip_grads'] = True # If true, clip gradients by norm 1. Default: True

SyntaxError: invalid syntax (<ipython-input-34-71e8e520c2df>, line 4)

Train our model.

In [ ]:
basicModel.train(gen, {'save_weights_path': '../weights/refactor_weights.npz', 'training_iters': 500})

### Plot results

Extract the next trial from the generator

In [ ]:
x,y,m = gen.next()


Plot the x value of the trial -- for the RDM, this includes two input neurons ith different coherence. TODO(jasmine): figure out axis scales.

In [ ]:
plt.plot(x[0,:,:])

Run the trained model on this trial (not included in the training set).

In [ ]:
plt.plot(basicModel.test(x)[0][0,:,:])

clean up the model to clear out the tensorflow namespace

In [ ]:
basicModel.destruct()